In [3]:
import json
import numpy as np


def calculate_polys(data: list[dict], x_label: str, y_label: str):
    freqs: list[str] = {p['freq'] for p in data}
    polys: dict[dict] = {}

    for freq in freqs:
        points = [p for p in data if p['freq'] == freq]

        x = [0, *[p[x_label] for p in points]]
        y = [0, *[p[y_label] for p in points]]
        # x = [p[x_label] for p in points]
        # y = [p[y_label] for p in points]

        power = 2
        temp = np.poly1d(np.polyfit(x, y, power))

        alpha = 'abcdefghijklmnopqrstuvwxyz'
        poly = {}
        for i, p in enumerate(range(power, -1, -1)):
            poly[alpha[i]] = round(temp[p], 10)

        polys[freq] = poly

    return polys


def create_poly_header(polys):
    header = []

    # make sure the outputs are in order
    bands = [p for p in polys['fwd']]
    bands.sort()

    header.append('polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['fwd'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    header.append('polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['rev'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    return ''.join(header)


with open('fwd.json', 'r') as f:
    fwd = json.load(f)['data']

with open('fwd_attenuated.json', 'r') as f:
    fwd.extend(json.load(f)['data'])

with open('rev.json', 'r') as f:
    rev = json.load(f)['data']

polys = {
    'fwd': calculate_polys(fwd, 't_fwd_volts', 'm_fwd'),
    'rev': calculate_polys(rev, 't_rev_volts', 'm_fwd'),
}


header = create_poly_header(polys)

print(header)

polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {
    {5.99246e-05, 0.0084949725, 0.0583486816}, // 01800000
    {5.1633e-05, 0.0078779723, 0.1638547607}, // 03500000
    {5.13935e-05, 0.0088719533, 0.0813424668}, // 07000000
    {5.44509e-05, 0.0081969944, 0.223598005}, // 10100000
    {5.34925e-05, 0.0103548966, 0.002006756}, // 14000000
    {5.25645e-05, 0.0107342809, -0.0772425672}, // 18068000
    {5.15167e-05, 0.0105767261, -0.113533868}, // 21000000
    {5.55405e-05, 0.0079429125, 0.240882941}, // 24890000
    {5.6346e-05, 0.0114048467, -0.1125376377}, // 28000000
    {8.83043e-05, 0.0083268183, 0.3591962366}, // 50000000
};

polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {
    {2.45485e-05, 0.0065363541, 0.0255649598}, // 01800000
    {2.12055e-05, 0.0063101253, 0.0243705659}, // 03500000
    {2.07476e-05, 0.0066377518, 0.0220154975}, // 07000000
    {2.24374e-05, 0.0068327958, 0.0266114382}, // 10100000
    {2.28101e-05, 0.0069334653, 0.0286353035}, // 14000000
    